In [ ]:
# default_exp core

# numbasom

> API details.

In [ ]:
#hide
from nbdev.showdoc import *
from numba import jit
import numpy as np
import math
import collections
from timeit import default_timer as timer

In [ ]:
#export
class NumbaSOM:
    """Instantiates the class with data"""
    def __init__(self, som_size, num_iterations, data, is_torus=False, isScaled=False):
        print ("hello")
        
    def say_hello(self, to):
        "say hello to somebody"
        return (f'Hello {to}!')

We can use this to say hello to anyone.

Test

In [ ]:
data = np.random.randn(100,3)

In [ ]:
som = NumbaSOM((20,20),1000,data)

hello


### To display class methods

In [ ]:
show_doc(NumbaSOM.say_hello)

<h4 id="NumbaSOM.say_hello" class="doc_header"><code>NumbaSOM.say_hello</code><a href="__main__.py#L7" class="source_link" style="float:right">[source]</a></h4>

> <code>NumbaSOM.say_hello</code>(**`to`**)

say hello to somebody

In [ ]:
som.say_hello("Nikola")

'Hello Nikola!'

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()